In [81]:
!pip install pydicom pylibjpeg SimpleITK numpy pillow

In [82]:
'**********************************************************************************************************************'
'This file contains the automatic method for epicardial fat segmentation'
'**********************************************************************************************************************'
import numpy as np
import cv2 as cv
from skimage.morphology import convex_hull_image

#MY FUNCTIONS
from algorithm.basicOperations import convert_image_to_rgb, add_images, otsu_mask, connect_components, segmentation, opening
from algorithm.findTemplate import find_template_sternal, find_template_spine, cut_image_from_top, cut_image_from_bottom
from algorithm.loadScan import load_scan
from algorithm.getPixelsHU import get_pixels_hu
from algorithm.pericardiumDelineation import pericardium_3points, ellipse, redefined_mask
from algorithm.contourAnalysis import draw_contours, moment
from algorithm.discardSlices import doTouchInMargin, countDiscardSlices
from algorithm.thrSegmentation import thrSegmentation
from algorithm.volume import volume
from algorithm.plot3D import plot_3d
from algorithm.sampleStack import sample_stack


In [215]:
def segmentEpicardialFat(DICOM_DATASET = 'path/dicom_file.dcm', OUTPUT_FOLDER='aux_img/'):
    'Select the patient dataset'
    patient = load_scan(DICOM_DATASET)

    'Thickness and pixel spacing'
    thickness = patient[0].SliceThickness
    print(thickness)
    px_spacing = patient[0].PixelSpacing[0]
    print(px_spacing)
    patient_id = patient[0].PatientID
    print(patient_id)
    no_slices = len(patient)
    print(no_slices)

    'Get image in HU scale'
    patients_hu = get_pixels_hu(patient)

    'Constants'
    WIDTH = patients_hu[0].shape[0]
    HEIGHT = patients_hu[0].shape[1]
    MID_HEIGHT = int(HEIGHT / 2)
    print(WIDTH, HEIGHT, MID_HEIGHT)

    ### CONSTANTS
    MAX_FAT = -30
    MIN_FAT = -200

    ### PRE-PROCESSING
    median = np.stack([cv.medianBlur(patient, 5) for patient in patients_hu])
    ### 1. ROI SELECTION
    'Matching template to detect sternal and spine'
    template_sternal = cv.imread('resources/template_sternum.png')
    template_spine = cv.imread('resources/template_spine.png')

    'Convert imagens to rgb channels'
    images_png = np.stack([convert_image_to_rgb(dicom, f'slices/{patient_id}_{i}', OUTPUT_FOLDER) for dicom, i in zip(median, range(0,no_slices))])

    top_points = find_template_sternal(images_png, template_sternal, method='cv.TM_CCOEFF_NORMED')[1]
    bottom_points = find_template_spine(images_png, template_spine, method='cv.TM_CCOEFF_NORMED')[0]

    'Remove the lungs'
    remove_lung_mask = np.stack([otsu_mask(patient) for patient in median])

    'Remove torax'
    remove_torax_mask = np.stack([cut_image_from_top(slice, point[1], WIDTH, y_reject=0) for slice, point in
                                  zip(remove_lung_mask, top_points)])

    'Remove spine'
    remove_spine_mask = np.stack(
        [cut_image_from_bottom(slice, MID_HEIGHT + point[1], WIDTH, y_reject=MID_HEIGHT) for slice, point in
         zip(remove_torax_mask, bottom_points)])

    ### 2. HEART ROI
    'Find contours with ellipse morphology'
    contours = np.stack([draw_contours(img.astype(np.uint8)) for img in remove_spine_mask])

    'Selection of the bigger component regarding the component with the heart'
    bigger_component = np.stack([connect_components(mask) for mask in contours])

    'Apply masks of ROI to segmentate the heart'
    masks = np.stack([contour > 0 for contour in bigger_component])
    heart = np.stack([segmentation(i, m) for i, m in zip(patients_hu, masks)])

    ### 3. PERICARDIUM DELIMITATION
    # -44 a 18 HU
    'Pericardium thresholding'
    pericardium_mask = np.stack([thrSegmentation(r, -44, -1) for r in heart])
    'Get pericardium contour'
    pericardium_contour = np.stack([convex_hull_image(mask) for mask in pericardium_mask])
    pericardium_opening = np.stack([opening(mask) for mask in pericardium_contour])

    'Redefine ROI of heart'
    new_contours = np.stack([contour > 0 for contour in pericardium_opening])

    'Discard slices that touch left or right margins'
    processed_slices = np.stack([doTouchInMargin(slice) for slice in new_contours])
    discard_slices = countDiscardSlices(processed_slices)

    'Convert imagens to rgb channels'
    np.stack([convert_image_to_rgb(cnt, f'contours/{patient_id}_{i}_c', OUTPUT_FOLDER) for cnt, i in zip(processed_slices, range(0,no_slices))])

    'Discard slices that touch left or right margins'
    new_masks = np.stack([contour > 0 for contour in processed_slices])
    new_heart = np.stack([segmentation(i, m) for i, m in zip(patients_hu, new_masks)])

    ### 4. EAT SEGMENTATION
    'Fat thresholding'
    fat_masks = np.stack([thrSegmentation(r, MIN_FAT, MAX_FAT) for r in new_heart])

    'To show the final result of segmentation'
    np.stack([convert_image_to_rgb(mask, f'fat/{patient_id}_{i}_fat', OUTPUT_FOLDER) for mask, i in zip(fat_masks, range(0, no_slices))])

    'Add slice image with segmentation mask'
    combined_images = np.stack([add_images(m, img) for m, img in zip(fat_masks, images_png)])

    'Save image with segmentation enhanced'
    np.stack([convert_image_to_rgb(img, f'combined/{patient_id}_{i}_combined', OUTPUT_FOLDER) for img, i in zip(combined_images, range(0,no_slices))])

    ### 6. VOLUME CALCULATION
    vol = volume(fat_masks, thickness, px_spacing)

    return patient_id, no_slices, vol



In [232]:
import SimpleITK as sitk
import os

def dicom_to_nifti(input_dicom, output_dicom_dir, write_nifti=False):
    input_dicom = "/Users/davideserra/Documents/Python383/HARTA-main/ScalarVolume_7"
    output_dicom_dir = './'
    # create the folder output_dicom_dir
    series_IDs = sitk.ImageSeriesReader.GetGDCMSeriesIDs(input_dicom)
    if not series_IDs:
        print("ERROR: given directory \"" + input_dicom + "\" does not contain a valid DICOM series.")

    itk_images = []
    filenames = []
    for i in range(0,len(series_IDs)):
        series_file_names = sitk.ImageSeriesReader.GetGDCMSeriesFileNames(input_dicom, series_IDs[i])
        filenames.append(series_file_names)
        series_reader = sitk.ImageSeriesReader()
        series_reader.SetFileNames(series_file_names)
        series_reader.MetaDataDictionaryArrayUpdateOn()
        series_reader.LoadPrivateTagsOn()
        image_dicom = series_reader.Execute()

        itk_images.append(image_dicom)
        if write_nifti:
            sitk.WriteImage(image_dicom, os.path.join(output_dicom_dir, series_IDs[i] + ".nii.gz"))

    return itk_images, filenames

In [235]:
import os
import numpy as np
import SimpleITK as sitk
from PIL import Image
import re

input_dicom_path = "/Users/davideserra/Documents/Python383/HARTA-main/ScalarVolume_7"
output_dicom_dir = './'

patient_id, no_slices, vol = segmentEpicardialFat(DICOM_DATASET = input_dicom_path,
                                                  OUTPUT_FOLDER=output_dicom_dir)

# 📂 Cartella con immagini PNG
input_folder = "/Users/davideserra/Documents/Python383/HARTA-main/fat"
#output_file = "patient8_EAT.nii.gz"

def extract_number(filename):
    match = re.search(r'OSPMBG_(\d+)_fat', filename)
    return int(match.group(1)) if match else -1

# 📥 Ordina e carica le immagini (assumiamo nomi ordinabili alfabeticamente)
image_files = sorted(
    [os.path.join(input_folder, f) for f in os.listdir(input_folder) if f.endswith(".png") and "OSPMBG_" in f],
    key=lambda f: extract_number(os.path.basename(f))
)

# 📊 Carica le immagini in array NumPy
slices = []
for filename in image_files:
    img = Image.open(filename).convert("L")  # "L" = scala di grigi
    img_arr = np.array(img)
    img_arr = np.where(img_arr == 255, 1, 0)
    slices.append(img_arr)

# 🧊 Stack lungo l'asse Z → (depth, height, width)
volume_array = np.stack(slices, axis=0)
print(volume_array.shape)


# 🔄 Converti in SimpleITK Image (NIfTI vuole (x, y, z))
volume_array = volume_array.astype(np.uint8)  # o .astype(np.int16) se necessario
print(volume_array.shape)
# volume_array = np.transpose(volume_array, (2, 1, 0))
print(volume_array.shape)
# volume_array = np.flip(volume_array, axis=0)
# volume_array = np.flip(volume_array, axis=1)
volume_sitk = sitk.GetImageFromArray(volume_array)

images, _ = dicom_to_nifti(input_dicom=input_dicom_path, 
                                   output_dicom_dir=output_dicom_dir,
                                   write_nifti=False)

ct = images[0]
# filename = filenames[0]
#
#ct = sitk.ReadImage("/Users/davideserra/Documents/Python383/HARTA-main/8.nii.gz")


# ℹ️ (opzionale) Aggiungi informazioni di spacing
# volume_sitk.SetSpacing([1.0, 1.0, 1.0])  # pixel spacing in mm (x, y, z)
volume_sitk.CopyInformation(ct)
# volume_sitk.SetSpacing(ct.GetSpacing())
# volume_sitk.SetOrigin(ct.GetOrigin())
# volume_sitk.SetDirection(ct.GetDirection())

# volume_sitk.SetSpacing( (1.0, 0.435546875, 0.435546875))
# volume_sitk.SetOrigin((-63.2822, -317.282, -302.125))
# volume_sitk.SetDirection((1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0))

output_file = "patient8_EAT.nii.gz"

# 💾 Salva come .nii.gz
sitk.WriteImage(volume_sitk, output_file, True)

print(f"✅ Salvato file NIfTI 3D: {output_file}")


0.5
.435546875
OSPMBG
275
512 512 256
(275, 512, 512)
(275, 512, 512)
(275, 512, 512)
✅ Salvato file NIfTI 3D: patient8_EAT.nii.gz


In [209]:
np.stack(slices).shape

(275, 512, 512)

In [199]:
volume_array.shape

(512, 512, 275)

In [192]:
volume_array.shape

(512, 512, 275)